In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pc

Check these links for climate related policies submitted by Viet Nam:

- https://www.climatewatchdata.org/countries/VNM?end_year=2022&start_year=1990&utm_source=chatgpt.com
- https://climateactiontracker.org/countries/vietnam/policies-action/

Data sources:
- Rain, Temperature, Climate Funds: https://data.vietnam.opendevelopmentmekong.net
- Development indicators, Fossil Fuel, Renewable Energy: https://data.worldbank.org
- SYB67: https://data.un.org
- CO2 emission by sectors: https://www.climatewatchdata.org/

### Environment

In [2]:
land = pd.read_csv("Datasets/SYB67_145_202411_Land.csv")
land = land[land['Region/Country/Area'] == 'Viet Nam']
land = land[["Year", "Series", "Value"]]
land = land.pivot(index="Series", columns="Year", values="Value")
land = land.sort_index(axis=1)
land

Year,2005,2010,2021,2023
Series,,,,
Arable land (% of total land area),20.3,20.5,21.7,NaN
Arable land (thousand hectares),"6,358","6,437","6,787",NaN
Forest cover (% of total land area),40.2,42.7,47.0,NaN
Forest cover (thousand hectares),"12,586","13,388","14,719",NaN
Important sites for terrestrial biodiversity protected (% of total sites protected),28.53,31.24,40.21,40.21
Land area (thousand hectares),"31,284","31,329","31,343",NaN
Permanent crops (% of total land area),9.8,11.7,15.7,NaN
Permanent crops (thousand hectares),"3,054","3,681","4,931",NaN


In [3]:
unit_rows = [
    "Arable land (thousand hectares)",
    "Forest cover (thousand hectares)",
    "Land area (thousand hectares)",
    "Permanent crops (thousand hectares)"
]

land_units = land.loc[unit_rows, [2005, 2010, 2021]]
land_units = (
    land_units
    .reset_index()
    .melt(id_vars="Series", var_name="Year", value_name="Value")
)
land_units['Value'] = land_units['Value'].str.replace(',', '').astype('float64')
land_units["Year"] = land_units["Year"].astype(str)

fig = px.bar(
    land_units,
    x="Year",
    y="Value",
    color="Series",
    barmode="group",
    title="Land Use (Thousand Hectares) Over Time"
)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero")
)
fig.write_html("gfx/land_use_abs.html")
fig.show()

In [4]:
unit_rows = [
    "Arable land (% of total land area)",
    "Forest cover (% of total land area)",
    "Permanent crops (% of total land area)"
]

land_per = land.loc[unit_rows, [2005, 2010, 2021]]
land_per = (
    land_per
    .reset_index()
    .melt(id_vars="Series", var_name="Year", value_name="Percent")
)
land_per['Percent'] = land_per['Percent'].str.replace(',', '').astype('float64')
land_per["Year"] = land_per["Year"].astype(str)
df_other = (
    land_per.groupby("Year")["Percent"]
    .sum()
    .reset_index()
    .rename(columns={"Percent":"sum_percent"})
)
df_other["Other (% of total land area)"] = 100 - df_other["sum_percent"]
df_other_long = df_other.melt(id_vars="Year", value_vars=["Other (% of total land area)"], var_name="Series", value_name="Percent")

df_pie = pd.concat([land_per, df_other_long], ignore_index=True)
fixed_order = [
    "Forest cover (% of total land area)",
    "Arable land (% of total land area)",
    "Permanent crops (% of total land area)",
    "Other land"
]

fig = px.pie(
    df_pie,
    names="Series",
    values="Percent",
    facet_col="Year",
    hole=0.3,
    title="Land Use (% of Total Land Area) by Year",
    category_orders={"Series": fixed_order}
)
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero")
)

fig.update_yaxes(rangemode="tozero")
fig.write_html("gfx/land_use_per.html")
fig.show()

In [5]:
species = pd.read_csv("Datasets/SYB67_313_202411_Threatened Species.csv")
species = species[species['Region/Country/Area'] == 'Viet Nam']
species = species[["Year", "Series", "Value"]]
species = species.pivot(index="Series", columns="Year", values="Value")
species = species.sort_index(axis=1)
species

Year,2004,2010,2015,2020,2021,2022,2023
Series,,,,,,,
Threatened Species: Invertebrates (number),0,92,127,156,156,191,191
Threatened Species: Plants (number),145,146,199,260,290,335,451
Threatened Species: Total (number),289,424,565,738,803,933,"1,053"
Threatened Species: Vertebrates (number),144,186,239,322,357,407,411


In [6]:
species_count = (
    species
    .reset_index()
    .melt(id_vars="Series", var_name="Year", value_name="Count")
)
species_count['Count'] = species_count['Count'].str.replace(',', '').astype('float64')
# species["Year"] = species["Year"].astype(str)
fixed_order = [
    "Threatened Species: Total (number)",
    "Threatened Species: Vertebrates (number)",
    "Threatened Species: Plants (number)",
    "Threatened Species: Invertebrates (number)"
]
fig = px.line(
    species_count,
    x="Year",
    y="Count",
    color="Series",
    title="Threatend Species Over Time",
    markers='.',
    category_orders={'Series': fixed_order}
)
fig.update_yaxes(rangemode="tozero")
fig.update_xaxes(dtick=1)
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero")
)
fig.write_html("gfx/species.html")
fig.show()

In [7]:
temp_rain = pd.read_csv("Datasets/precipitation_mean_max_min_temp_1901_2020_en.csv") # data for a cycle of 30 years
temp_rain.dropna()

,Month,Min-Temperature,Mean-Temperature,Max-Temperature,Precipitation,Period,Level
0,Jan,15.86,19.77,23.72,33.59,1901-1930,1.0
1,Feb,16.79,20.69,24.63,28.86,1901-1930,2.0
2,Mar,16.74,22.18,27.68,42.81,1901-1930,3.0
3,Apr,21.32,25.38,29.48,67.88,1901-1930,4.0
4,May,23.07,26.96,30.90,176.59,1901-1930,5.0
5,Jun,23.53,27.18,30.88,227.04,1901-1930,6.0
6,Jul,23.71,27.17,30.67,256.11,1901-1930,7.0
7,Aug,23.45,26.97,30.55,264.96,1901-1930,8.0
8,Sep,22.71,26.20,29.74,274.86,1901-1930,9.0
9,Oct,21.10,24.69,28.33,228.44,1901-1930,10.0


In [13]:
year = '1991-2020'
df = temp_rain[temp_rain['Period'] == year]
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=df["Month"],
        y=df["Max-Temperature"],
        mode='lines',
        line=dict(color='lightcoral'),
        name='Max Temp'
    )
)

fig.add_trace(
    go.Scatter(
        x=df["Month"],
        y=df["Min-Temperature"],
        mode='lines',
        line=dict(color='lightcoral'),
        fill='tonexty',
        name='Min Temp'
    )
)

fig.add_trace(
    go.Scatter(
        x=df["Month"],
        y=df["Mean-Temperature"],
        mode='lines+markers',
        line=dict(color='red', width=2),
        name='Mean Temp'
    )
)

fig.add_trace(
    go.Scatter(
        x=df["Month"],
        y=df["Precipitation"],
        mode='lines+markers',
        line=dict(color='blue', dash='dash'),
        name='Precipitation',
    ),
    secondary_y=True
)

fig.update_layout(
    title=f"Monthly Temperature Range & Precipitation over Period {year}",
    xaxis_title="Month",
    yaxis_title="Temperature (°C)",
    legend=dict(x=0.01, y=0.99)
)

fig.update_yaxes(title_text="Temperature (°C)", secondary_y=False, range=[0, 40])
fig.update_yaxes(title_text="Precipitation (mm)", secondary_y=True, range=[0, 300])
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero")
)
fig.write_html("gfx/weather_9120.html")
fig.show()

In [14]:
temp_change = pd.read_csv("Datasets/Vietnam Cities Temp Change.csv", header=None)
years = temp_change.iloc[0, 1:].tolist() 
months = temp_change.iloc[1, 1:].tolist()  
cities = temp_change.iloc[2:, 0].tolist()  
values = temp_change.iloc[2:, 1:].replace('..', '0').astype(float).values
n_cities = len(cities)
n_columns = len(years)

df_long = pd.DataFrame({
    'City': np.repeat(cities, n_columns),
    'Year': np.tile(years, n_cities),
    'Month': np.tile(months, n_cities),
    'Temperature': values.flatten()
})
df_long

,City,Year,Month,Temperature
0,Lai Chau,2002,January,16.7
1,Lai Chau,2002,February,19.5
2,Lai Chau,2002,March,23.6
3,Lai Chau,2002,April,25.6
4,Lai Chau,2002,May,26.0
...,...,...,...,...
2875,Ca Mau,2017,August,27.7
2876,Ca Mau,2017,September,28.2
2877,Ca Mau,2017,October,27.5
2878,Ca Mau,2017,November,27.7


In [16]:
from geopy.geocoders import Nominatim
import folium
from branca.colormap import linear

# City name mapping for geocoding
CITY_NAME_MAP = {
    "Ha Noi": "Hanoi",
    "Bai Chay": "Ha Long",
    "Ca Mau": "Ca Mau City"
}

# Initialize geocoder once
geolocator = Nominatim(user_agent="vietnam_map_generator")

def create_vietnam_temp_map(df, year, month, filename=None):
    """
    Generate a Folium map showing temperatures for Vietnamese cities for a given year and month.
    
    Parameters:
        df : pandas.DataFrame
            Must contain columns: ['City', 'Year', 'Month', 'Temperature']
        year : int
            The year to filter data
        month : int or str
            The month to filter data (1-12 or month name)
        filename : str, optional
            If provided, saves the map as an HTML file with this name
    Returns:
        folium.Map object
    """
    # Filter data for the specified month and year
    df_filtered = df[(df['Year'] == year) & (df['Month'] == month)]
    
    if df_filtered.empty:
        print(f"No data found for {month}-{year}")
        return None

    
    # Create city -> temp dictionary
    cities_temp = {}
    for _, row in df_filtered.iterrows():
        city = CITY_NAME_MAP.get(row['City'], row['City'])  # map names if needed
        cities_temp[city] = row['Temperature']
    
    # Create base map
    m = folium.Map(location=[16.0, 108.0], zoom_start=6)
    
    # Color scale based on temperatures
    colormap = linear.YlOrRd_09.scale(0, 40)
    colormap.caption = f"Temperature (°C) - {month}-{year}"
    colormap.add_to(m)
    
    # Add markers
    for city, temp in cities_temp.items():
        location = geolocator.geocode(f"{city}, Vietnam", timeout=15)
        if location:
            color = colormap(temp)
            folium.CircleMarker(
                location=[location.latitude, location.longitude],
                radius=8,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.8,
                popup=f"{city}: {temp}°C"
            ).add_to(m)
    
    # Save if filename is provided
    if filename:
        m.save(filename)
        print(f"Map saved as {filename}")
    
    return m

years = range(2002, 2018)
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for i in years:
    for j in months:
        create_vietnam_temp_map(df_long, year=f'{i}', month=j, filename=f"gfx/temp_cities_{j}_{i}.html")

Map saved as gfx/temp_cities_January_2002.html
Map saved as gfx/temp_cities_February_2002.html
Map saved as gfx/temp_cities_March_2002.html
Map saved as gfx/temp_cities_April_2002.html
Map saved as gfx/temp_cities_May_2002.html
Map saved as gfx/temp_cities_June_2002.html
Map saved as gfx/temp_cities_July_2002.html
Map saved as gfx/temp_cities_August_2002.html
Map saved as gfx/temp_cities_September_2002.html
Map saved as gfx/temp_cities_October_2002.html
Map saved as gfx/temp_cities_November_2002.html
Map saved as gfx/temp_cities_December_2002.html
Map saved as gfx/temp_cities_January_2003.html
Map saved as gfx/temp_cities_February_2003.html
Map saved as gfx/temp_cities_March_2003.html
Map saved as gfx/temp_cities_April_2003.html
Map saved as gfx/temp_cities_May_2003.html
Map saved as gfx/temp_cities_June_2003.html
Map saved as gfx/temp_cities_July_2003.html
Map saved as gfx/temp_cities_August_2003.html
Map saved as gfx/temp_cities_September_2003.html
Map saved as gfx/temp_cities_Octobe

### Energy

In [17]:
energy_trade = pd.read_csv("Datasets/SYB67_263_202411_Production, Trade and Supply of Energy.csv")
energy_trade = energy_trade[energy_trade['Region/Country/Area'] == 'Viet Nam']
energy_trade = energy_trade[["Year", "Series", "Value"]]
energy_trade = energy_trade.pivot(index="Series", columns="Year", values="Value")
energy_trade = energy_trade.sort_index(axis=1)
energy_trade

Year,1995,2000,2005,2010,2015,2019,2020,2021
Series,,,,,,,,
Changes in stocks (petajoules),20,50,70,5,74,15,-53,86
Net imports [Imports - Exports - Bunkers] (petajoules),-207,-421,-786,-409,247,"1,588","1,937","1,446"
Primary energy production (petajoules),"1,136","1,733","2,612","2,747","3,035","2,643","2,337","2,602"
Supply per capita (gigajoules),12,16,21,27,35,44,45,41
Total supply (petajoules),908,"1,262","1,756","2,334","3,207","4,215","4,327","3,961"


In [18]:
keep_rows = [
    "Net imports [Imports - Exports - Bunkers] (petajoules)",
    "Total supply (petajoules)"
]

energy_trade_plot = energy_trade.loc[keep_rows]
energy_trade_plot = (
    energy_trade_plot
    .reset_index()
    .melt(id_vars="Series", var_name="Year", value_name="Petajoules")
)
energy_trade_plot['Petajoules'] = energy_trade_plot['Petajoules'].str.replace(',', '').astype('float64')
# energy_trade["Year"] = energy_trade["Year"].astype(str)

fig = px.line(
    energy_trade_plot,
    x="Year",
    y="Petajoules",
    color="Series",
    title="Energy Import and Supply Over Time",
    markers=True,
    line_shape='spline',
)
fig.update_xaxes(dtick=1)
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero"),
)
fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgrey',  # color for all grid lines, including zero line
    zeroline=True,           # keep zero line
    zerolinecolor='lightgrey',  # match grid
    zerolinewidth=1,
    rangemode="tozero"
)
fig.write_html('gfx/energy_trade.html')
fig.show()

In [19]:
fossil = pd.read_csv("Datasets/Fossil fuel energy portion.csv")
fossil = fossil[fossil['Country Name'] == 'Viet Nam']
fossil = fossil.dropna(axis=1)
fossil

,Country Name,Country Code,Indicator Name,Indicator Code,1990,1991,1992,1993,1994,1995,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
257,Viet Nam,VNM,Fossil fuel energy consumption (% of total),EG.USE.COMM.FO.ZS,27.62,26.82,28.1,30.36,33.61,37.03,...,68.17,70.64,78.72,78.7,76.93,80.13,83.72,84.78,79.81,79.35


In [20]:
renew = pd.read_csv("Datasets/Renewable energy portion.csv")
renew = renew[renew['Country Name'] == 'Viet Nam']
renew = renew.dropna(axis=1)
renew

,Country Name,Country Code,Indicator Name,Indicator Code,1990,1991,1992,1993,1994,1995,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
257,Viet Nam,VNM,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,75.9,75.5,74.5,70.6,67.8,64.9,...,37.9,37.4,36.7,27.8,26.8,28.3,24.3,20.4,18.9,24.2


In [21]:
years = [str(y) for y in range(1990, 2022)]

df_fossil_long = fossil.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=years,
    var_name="Year",
    value_name="Fossil"
)

df_renew_long = renew.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    value_vars=years,
    var_name="Year",
    value_name="Renewable"
)

df_combined = pd.merge(
    df_fossil_long[['Year', 'Fossil']],
    df_renew_long[['Year', 'Renewable']],
    on="Year"
)

df_combined['Year'] = df_combined['Year'].astype(int)
df_plot = df_combined.melt(id_vars='Year', var_name='Energy Type', value_name='Percentage')

fig = px.bar(
    df_plot,
    x='Energy Type',
    y='Percentage',
    color='Energy Type',
    animation_frame='Year',
    range_y=[0, 100],
    title='Energy Consumption Over Time',
    text='Percentage'
)

fig.update_layout(
    yaxis_title='Percentage of Total Energy',
    xaxis_title='Energy Type',
    showlegend=False
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero"),
)
fig.write_html('gfx/energy_type.html')
fig.show()

In [ ]:
# no longer in use

co2 = pd.read_csv("Datasets/SYB67_310_202411_Carbon Dioxide Emission Estimates.csv")
co2 = co2[co2['Region/Country/Area'] == 'Viet Nam']
co2 = co2[["Year", "Series", "Value"]]
co2 = co2.pivot(index="Series", columns="Year", values="Value")
co2 = co2.sort_index(axis=1)
co2

Year,1975,1985,2005,2010,2015,2019,2020,2021
Series,,,,,,,,
Emissions (thousand metric tons of carbon dioxide),"20,657","19,451","80,541","118,225","181,502","290,178","311,061","258,558"
Emissions per capita (Kilograms of carbon dioxide),0.4,0.3,1.0,1.4,2.0,3.0,3.2,2.7


In [22]:
co2_by_sector = pd.read_csv("Datasets/ghg-emissions.csv")
co2_by_sector = co2_by_sector[co2_by_sector['Sector'].isin(['Energy','Industrial Processes', 'Land-Use Change and Forestry'])]
co2_by_sector

,Sector,unit,1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Energy,MtCO2e,18.19,18.43,19.01,22.37,24.60,28.83,32.16,37.37,...,127.27,133.85,147.84,167.00,185.71,187.64,237.44,285.50,295.59,282.11
1,Industrial Processes,MtCO2e,1.12,1.33,1.77,1.85,2.06,2.54,2.85,3.45,...,28.12,30.32,32.69,34.30,36.82,42.74,49.64,56.54,60.07,62.07
4,Land-Use Change and Forestry,MtCO2e,-73.58,-73.58,-73.58,-73.58,-73.58,-73.56,-73.60,-73.67,...,-23.81,-21.77,-23.18,-23.81,-13.46,-13.52,-13.51,-13.50,-13.10,-13.49


In [23]:
years = co2_by_sector.columns[2:]

fig = go.Figure()

for i, row in co2_by_sector.iterrows():
    fig.add_trace(go.Bar(
        x=years,
        y=row[2:].astype('float64'),  # all year values for this sector
        name=row['Sector']
    ))

fig.update_layout(
    barmode='relative',
    title='CO2 Emissions by Sector',
    xaxis_title='Year',
    yaxis_title='MtCO2e',
    legend_title='Sector'
)
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero")
)
fig.write_html('gfx/co2_sector.html')
fig.show()

### Development Indicators

In [24]:
# idx of Vietnam, East Asia & Pacific, and World

dev_idx = pd.read_csv("Datasets/Development Indicators VN and Others.csv")
# dev_idx.dropna()
id_vars = ["Country Name", "Country Code", "Series Name", "Series Code"]
year_cols = [c for c in dev_idx.columns if c not in id_vars]
df_long = dev_idx.melt(
    id_vars=id_vars,
    value_vars=year_cols,
    var_name="Year",
    value_name="Value"
)

df_long["Year"] = df_long["Year"].str.extract(r"(\d{4})").astype(float)
df_long["Value"] = pd.to_numeric(df_long["Value"], errors="coerce")

dev_idx = (
    df_long
    .dropna(subset=["Year"])  # ensure years are valid
    .groupby(["Series Name", "Country Name"])
    .apply(lambda x: {int(y): v for y, v in zip(x["Year"], x["Value"]) if not np.isnan(v)})
    .unstack("Country Name")
)

dev_idx

C:\Users\KHANH LINH\AppData\Local\Temp\ipykernel_7688\769933118.py:21: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Country Name,East Asia & Pacific,East Asia & Pacific (excluding high income),Viet Nam,World
Series Name,,,,
Access to clean fuels and technologies for cooking (% of population),"{2000: 41.6876632107138, 2001: 42.242526367174...","{2000: 35.2407805735228, 2001: 35.883830278608...","{2000: 13.6, 2001: 16.5, 2002: 19.3, 2003: 22....","{2000: 49.1666460883022, 2001: 49.686345881075..."
Access to electricity (% of population),"{2000: 92.0618144338661, 2001: 92.543798227047...","{2000: 91.1381444668449, 2001: 91.679807393445...","{1997: 78.4, 1998: 86.1759872436523, 1999: 86....","{1998: 73.196235388656, 1999: 74.5032105116741..."
Adjusted net national income per capita (current US$),"{1975: 548.256824727202, 1976: 590.65394601849...","{1975: 167.512313009585, 1976: 164.56545661720...","{1989: 92.975226653232, 1990: 83.9230358669634...","{1975: 1219.20564408735, 1976: 1299.1960477747..."
Carbon dioxide (CO2) emissions from Building (Energy) (Mt CO2e),"{1975: 491.7095, 1976: 502.5157, 1977: 550.330...","{1975: 326.268, 1976: 328.394, 1977: 367.5165,...","{1975: 12.8191, 1976: 8.4934, 1977: 9.2946, 19...","{1975: 3121.8512423211, 1976: 3248.5882000116,..."
Carbon dioxide (CO2) emissions from Industrial Combustion (Energy) (Mt CO2e),"{1975: 921.6295, 1976: 963.2635, 1977: 1030.59...","{1975: 578.0067, 1976: 612.9176, 1977: 691.172...","{1975: 0.0287, 1976: 0.1203, 1977: 0.0602, 197...","{1975: 3715.2006889625, 1976: 3883.827958798, ..."
Carbon dioxide (CO2) emissions from Transport (Energy) (Mt CO2e),"{1975: 292.8977, 1976: 303.4062, 1977: 325.898...","{1975: 108.1979, 1976: 109.7396, 1977: 118.623...","{1975: 2.2849, 1976: 0.7616, 1977: 0.4874, 197...","{1975: 3285.8173799409, 1976: 3422.0904083995,..."
Energy use (kg of oil equivalent per capita),"{1975: 725.633700098986, 1976: 748.01130423944...","{1975: 494.888324827163, 1976: 503.46001190520...","{1975: 299.19148982584, 1976: 261.781106488337...","{1975: 1366.92673491981, 1976: 1421.2687382273..."
"Exports of goods, services and primary income (BoP, current US$)","{1989: 690585007341.248, 1990: 788571999067.91...","{1982: 82377990339.1063, 1983: 82778800556.735...","{1996: 9638000000.0, 1997: 11851000000.0, 1998...","{1981: 2874059398644.34, 1982: 2707620812026.2..."
"Imports of goods, services and primary income (BoP, current US$)","{1989: 709427942177.137, 1990: 794338749163.48...","{1982: 94528228428.3779, 1983: 98710389105.662...","{1996: 12858000000.0, 1997: 14264000000.0, 199...","{1979: 2264344849540.35, 1980: 2786504972652.4..."


In [28]:
# choose 1 index to visualize: 0, 1, 2, 6
row = dev_idx.iloc[6]
feature_name = row.name


regions = dev_idx.columns

# Prepare a dictionary to hold parsed data
data_dict = {}

for region in regions:
    cell = row[region]
    if isinstance(cell, str):
        # Convert string representation of dict to actual dict
        cell_dict = ast.literal_eval(cell)
    else:
        cell_dict = cell
    # Convert keys and values to lists
    years = list(cell_dict.keys())
    values = list(cell_dict.values())
    data_dict[region] = pd.Series(values, index=years, dtype=float)

# Combine into a single DataFrame
plot_df = pd.DataFrame(data_dict)

order = [
    'World',
    'East Asia & Pacific',
    'East Asia & Pacific (excluding high income)',
    'Viet Nam'
]
plot_df = plot_df[[col for col in order if col in plot_df.columns]]

# Plot with Plotly
fig = go.Figure()

colors = pc.qualitative.Plotly  # A palette of 10 distinct colors

highlight = 'Viet Nam'  # example
for region in plot_df.columns:
    fig.add_trace(go.Scatter(
        x=plot_df.index,
        y=plot_df[region],
        mode='lines+markers',
        name=region,
        line=dict(width=3 if region == highlight else 1),
        opacity=1.0 if region == highlight else 0.6
    ))

fig.update_layout(
    title=feature_name,
    xaxis_title='Year',
    yaxis_title='Value',
)
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey', rangemode="tozero")
)
fig.write_html('gfx/dev_idx_6.html')
fig.show()

In [2]:
data = [
    (1992, "Signed UNFCCC",
     "Vietnam signed the UN Framework Convention on Climate Change, joining global climate efforts."),
    (1994, "Ratified UNFCCC",
     "Vietnam became an official Party to the UNFCCC, making climate cooperation legally binding."),
    (2002, "Ratified Kyoto Protocol",
     "Joined the Kyoto Protocol framework, enabling participation in international GHG mechanisms."),
    (2008, "National Target Programme to Respond to Climate Change",
     "First national program to assess climate impacts and mainstream adaptation and mitigation."),
    (2011, "National Strategy on Climate Change",
     "Set long-term direction to 2050 for adaptation, mitigation, and sectoral priorities."),
    (2012, "National Green Growth Strategy",
     "Promoted low-carbon economic growth, energy efficiency, and green investment."),
    (2014, "Law on Environmental Protection 2014",
     "Strengthened environmental assessment and created foundations for climate policy."),
    (2015, "Paris Agreement Signed",
     "Vietnam submitted its INDC and joined the global Paris climate framework."),
    (2016, "Ratified Paris Agreement",
     "Vietnam formally ratified the Paris Agreement and issued a domestic action plan."),
    (2020, "Law on Environmental Protection 2020",
     "Broadened legal basis for GHG mitigation and carbon market development (effective 2022)."),
    (2022, "Decree 06/2022/ND-CP",
     "Implemented GHG mitigation, inventories, and roadmap for domestic carbon market."),
    (2023, "Power Development Plan VIII (PDP8)",
     "Set renewable targets (47% by 2030) and reshaped Vietnam's energy structure."),
    (2025, "Decree 119/2025/ND-CP",
     "Amended Decree 06/2022, strengthening carbon market and enforcement (effective Aug 2025)."),
    (2025, "Pilot Emissions Trading Scheme (ETS)",
     "Launched pilot ETS for steel, cement, and thermal power sectors (2025-2029)."),
]

df = pd.DataFrame(data, columns=["Year", "Milestone", "Description"])
colors = px.colors.qualitative.Dark24[:len(df)]

# --- Create figure ---
fig = go.Figure()

# Baseline using a line shape
fig.add_shape(type="line",
              x0=1990, y0=1, x1=2026, y1=1,
              line=dict(color="gray", width=2, dash="dash"),
              layer="below")

# Add a triangle at the end to simulate an arrow
fig.add_trace(go.Scatter(
    x=[2026],
    y=[1],
    mode="markers",
    marker=dict(symbol="triangle-right", size=15, color="gray"),
    hoverinfo="skip"
))

for i, row in df.iterrows():
    fig.add_trace(go.Scatter(
        x=[row["Year"]],
        y=[1],  # horizontal timeline
        mode="markers",
        marker=dict(
            size=30,
            color=colors[i],
            line=dict(width=2, color="DarkSlateGrey")
        ),
        hovertemplate=f"<b>{row['Milestone']}</b><br><br>"
                      f"<i>Year:</i> {row['Year']}<br>{row['Description']}<extra></extra>"
    ))

# --- Layout ---
fig.update_yaxes(visible=False)
fig.update_layout(
    title=dict(
        text="Vietnam Climate Policy Milestones (1992-2025)",
        x=0.025,  # left-aligned with padding
        xanchor="left",
        yanchor="top"
    ),
    annotations=[
        dict(
            text="(Hover over each milestone for more details)",
            x=0.0005,
            y=1.07,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=12, color="gray"),
            align="left"
        )
    ],
    showlegend=False,
    height=450,
    margin=dict(l=40, r=40, t=80, b=40),
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(title="Year", dtick=2, showgrid=True, gridcolor='lightgrey', range=[1989.5, 2026.5]),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)
fig.update_layout(
    title_font=dict(size=24, family="Arial Black"),
    plot_bgcolor="#f5f5f5",
    paper_bgcolor="#f5f5f5",
    font=dict(size=14),
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey')
)
fig.write_html("gfx/policies_timeline.html")
fig.show()